In [0]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [0]:
import numpy as np
import pandas as pd
import io
dataset = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))

In [0]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
dataset.shape

(10000, 14)

In [0]:
dataset['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [0]:
X = dataset.iloc[:,3:13]
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [0]:
y = dataset.iloc[:,13]
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [0]:
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [0]:
geography.head()

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1


In [0]:
X = pd.concat([X, geography, gender], axis = 1)

In [0]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


In [0]:
X.drop(['Geography', 'Gender'], axis = 1, inplace=True)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Building Artificial Neural Network (ANN)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout

In [0]:
# Initialising ANN

classifier = Sequential()

In [0]:
# Adding input layer 
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform', activation='relu', input_dim = 11))

# we can add dropout layer after each hidden layer
# classifier.add(Dropout(0.3))

# Adding first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform', activation='relu'))

# we can add dropout layer after each hidden layer
# classifier.add(Dropout(0.3))

# Adding output layer
classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))


In [0]:
# Compiling the ANN

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
# Fitting ANN to the training set

model_hist = classifier.fit(X_train, y_train, validation_split=0.33, batch_size=10, nb_epoch = 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 5359 samples, validate on 2641 samples
Epoch 1/100
5359/5359 [==============================] - 1s 228us/step - loss: 0.5572 - accuracy: 0.7514 - val_loss: 0.4940 - val_accuracy: 0.7967
Epoch 2/100
5359/5359 [==============================] - 1s 154us/step - loss: 0.4701 - accuracy: 0.8050 - val_loss: 0.4653 - val_accuracy: 0.8080
Epoch 3/100
5359/5359 [==============================] - 1s 154us/step - loss: 0.4394 - accuracy: 0.8186 - val_loss: 0.4431 - val_accuracy: 0.8171
Epoch 4/100
5359/5359 [==============================] - 1s 152us/step - loss: 0.4174 - accuracy: 0.8261 - val_loss: 0.4290 - val_accuracy: 0.8194
Epoch 5/100
5359/5359 [==============================] - 1s 152us/step - loss: 0.4013 - accuracy: 0.8311 - val_loss: 0.4121 - val_accuracy: 0.8307
Epoch 6/100
5359/5359 [==============================] - 1s 154us/step - loss: 0.3871 - accuracy: 0.8414 - val_loss: 0.3995 - val_accuracy: 0.8364
Epoch 7/100
5359/5359 [==============================] - 1s 153us/step

In [0]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
cm

array([[1527,   68],
       [ 209,  196]])

In [0]:
score = accuracy_score(y_pred, y_test)
score

0.8615